In [1]:
from google.colab import drive
drive.mount('/content/drive')
% cd '/content/drive/My Drive/freq_bias'

Mounted at /content/drive
/content/drive/My Drive/freq_bias


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr 22 14:53:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
% pip install foolbox==2.3.0

In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import pickle
# import foolbox
import csv
import pdb
import time
import ast

from PIL import Image
from matplotlib import pyplot as plt
from torchvision import datasets
from torchvision import models as tv
from dlib import get_face_chip
from torch.utils.data import DataLoader, Dataset

from utils import *
from configs import configs

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
train_dataset = FairFaceDataset('../fairface/fairface_label_train.csv', attr='race')
train_dataloader = DataLoader(
    train_dataset,    # The dataset
    batch_size=128,   # Batch size
    shuffle=True,      # Shuffles the dataset at every epoch
    pin_memory=True,   # Copy data to CUDA pinned memory
                       # so that they can be transferred to the GPU very fast
    num_workers=16      # Number of worker processes for loading data.
                       # If zero, use the current process (blocks until data are loaded)
                       # Otherwise fork/spawn new processes (asynchronous load)
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [6]:
val_dataset = FairFaceDataset('../fairface/fairface_label_val.csv', mode='val', attr='race')
val_dataloader = DataLoader(
    val_dataset,    # The dataset
    batch_size=64,      # Batch size
    shuffle=True,      # Shuffles the dataset at every epoch
    pin_memory=True,   # Copy data to CUDA pinned memory
                       # so that they can be transferred to the GPU very fast
    num_workers=8      # Number of worker processes for loading data.
                       # If zero, use the current process (blocks until data are loaded)
                       # Otherwise fork/spawn new processes (asynchronous load)
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [5]:
train_dataset = UTKDataset('../utkface/utkface_train.csv', attr='race')
train_dataloader = DataLoader(
    train_dataset,    # The dataset
    batch_size=64,    # Batch size
    shuffle=True,    # Shuffles the dataset at every epoch
    pin_memory=True,   # Copy data to CUDA pinned memory
                       # so that they can be transferred to the GPU very fast
    num_workers=16      # Number of worker processes for loading data.
                       # If zero, use the current process (blocks until data are loaded)
                       # Otherwise fork/spawn new processes (asynchronous load)
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [6]:
val_dataset = UTKDataset('../utkface/utkface_test.csv', mode='val', attr='race')
val_dataloader = DataLoader(
    val_dataset,    # The dataset
    batch_size=64,      # Batch size
    shuffle=True,      # Shuffles the dataset at every epoch
    pin_memory=True,   # Copy data to CUDA pinned memory
                       # so that they can be transferred to the GPU very fast
    num_workers=8      # Number of worker processes for loading data.
                       # If zero, use the current process (blocks until data are loaded)
                       # Otherwise fork/spawn new processes (asynchronous load)
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
val_dataset = SkinToneDataset()
val_dataloader = DataLoader(
    val_dataset,    # The dataset
    batch_size=64,      # Batch size
    shuffle=True,      # Shuffles the dataset at every epoch
    pin_memory=True,   # Copy data to CUDA pinned memory
                       # so that they can be transferred to the GPU very fast
    num_workers=8      # Number of worker processes for loading data.
                       # If zero, use the current process (blocks until data are loaded)
                       # Otherwise fork/spawn new processes (asynchronous load)
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
! ls ../freq_bias/models/

Fairface_Multi_layer0_11_layerall_3_epoch_13_trial1.pt
Fairface_Multi_layer0_11_layerall_3_epoch_13_trial2.pt
Fairface_Multi_layer0_11_layerall_3_epoch_13_trial3.pt
Fairface_Multi_layer0_11_layerall_7_epoch_13_trial1.pt
Fairface_Multi_layer0_13_layerall_3_epoch_13_trial1.pt
Fairface_Multi_layer0_13_layerall_3_epoch_13_trial2.pt
Fairface_Multi_layer0_13_layerall_3_epoch_13_trial3.pt
Fairface_Multi_layer0_13_layerall_7_epoch_13_trial1.pt
Fairface_Multi_layer0_3_layerall_3_epoch_13_trial1.pt
Fairface_Multi_layer0_3_layerall_3_epoch_13_trial2.pt
Fairface_Multi_layer0_3_layerall_3_epoch_13_trial3.pt
Fairface_Multi_layer0_3_layerall_7_epoch_13_trial1.pt
Fairface_Multi_layer0_5_layerall_3_epoch_13_trial1.pt
Fairface_Multi_layer0_5_layerall_3_epoch_13_trial2.pt
Fairface_Multi_layer0_7_layerall_3_epoch_13_trial1.pt
Fairface_Multi_layer0_7_layerall_3_epoch_13_trial2.pt
Fairface_Multi_layer0_7_layerall_3_epoch_13_trial3.pt
Fairface_Multi_layer0_7_layerall_7_epoch_13_trial1.pt
Fairface_Multi_layer

In [8]:
class Trainer():

    def __init__(self, model, optimizer=None, scheduler=None, train_dataloader=None, val_dataloader=None, attr='age', load_path=None, save_path=None, dataset='fairface', attr_masks=None):
      self.model = model.to(device)
      self.optimizer = optimizer
      self.scheduler=scheduler
      self.train_dataloader = train_dataloader
      self.val_dataloader = val_dataloader
      self.criterion = nn.CrossEntropyLoss()
      self.save_path = save_path
      self.dataset = dataset
      self.attr_masks = attr_masks

      if self.dataset == 'fairface':
        self.ages = [ '0-2', 
              '3-9',
              '10-19',
              '20-29',
              '30-39',
              '40-49',
              '50-59',
              '60-69',
              'more than 70',
              ]

        self.races = ['East Asian',
              'White',
              'Latino_Hispanic',
              'Southeast Asian',
              'Black',
              'Indian',
              'Middle Eastern']

      if self.dataset == 'utkface':
        self.ages = [ '0-9', 
                '10-19',
                '20-29',
                '30-39',
                '40-49',
                '50-59',
                '60-69',
                'more than 70',]

        self.races = ['White',
                'Black', 
                'Asian', 
                'Indian', 
                'Others']

      self.skins = ['light',
              'dark']

      self.attr = self.ages if attr == 'age' else self.races

      # self.attr = self.skins

    def save_model(self, path):
      torch.save(self.model, path)

    def get_confusion(self):
      self.model.eval()

      running_corrects = {i:{0:0, 1:0} for i in self.attr}
      running_all = {i:{0:0, 1:0} for i in self.attr}

      for i, (data, age_labels, gender_labels, race_labels, t) in enumerate(self.val_dataloader):

        torch.cuda.empty_cache()

        data = data.to(device)
        gender_labels = gender_labels.squeeze().to(device)

        with torch.no_grad():
          outputs = self.model(data)

        _, preds = torch.max(outputs, 1)

        running_corrects, running_all = self._update_confusion(running_corrects, running_all, preds, gender_labels.data, t)
        corrects = torch.sum(preds == gender_labels.data)

        if i % 40 == 0:
          print("Iteration {}".format(i))
          print("*"*20)

        del data, gender_labels, preds

      confusion_matrix = []
      for i in self.attr:
        true0 = running_corrects[i][0]
        true1 = running_corrects[i][1]
        false0 = running_all[i][0] - running_corrects[i][0]
        false1 = running_all[i][1] - running_corrects[i][1]
        confusion_matrix.append([[true0, false0],
                    [false1, true1]])
      return confusion_matrix

    def _update_confusion(self, running_corrects, running_all, preds, label, t):
      for i in range(len(t)):
        running_all[t[i]][label[i].item()] += 1
        if preds[i] == label[i]:
          running_corrects[t[i]][label[i].item()] += 1
      return running_corrects, running_all

    def get_masked_input(self, input, t):
      output = []

      for i in range(len(t)):
        tmp = self._get_fourier(input[i])
        # tmp1 = np.multiply(self.attr_masks[t[i]], tmp)
        tmp1 = np.multiply(self.attr_masks['White'], tmp)
        tmp2 = self._inverse_fft(tmp1)
        output.append(tmp2)
      return torch.tensor(output, dtype=torch.float).to(device)
    
    def get_modified_input(self, input, corr_array):
      f_ = self._get_fourier(input)

      for ary in corr_array:
        f_[:, :, int(ary[0][0]), int(ary[0][1])] += ary[1][0] + ary[1][1] * 1j
      tmp = torch.tensor(self._inverse_fft(f_), dtype=torch.float)
  
      return tmp.to(device)

    def _get_fourier(self, imag):
      image_sample = np.fft.fft2(imag)
      F_orig1 = np.fft.fftshift(image_sample)
      return F_orig1

    def _inverse_fft(self, s):
      fft_img_mod = np.fft.ifftshift(s)
      img_mod = np.fft.ifft2(fft_img_mod)
      img_mod = img_mod.real

      return img_mod

    def eval_attr(self, mask=False, modified=False, corr_array=None, save_path=None):
      self.model.eval()

      running_corrects = {i:0 for i in self.attr}
      running_all = {i:0 for i in self.attr}

      for i, (data, age_labels, gender_labels, race_labels, t) in enumerate(self.val_dataloader):

        torch.cuda.empty_cache()

        data = data.to(device)
        gender_labels = gender_labels.squeeze().to(device)

        if mask:
          data = self.get_masked_input(data.cpu(), t)

        if modified:
          data = self.get_modified_input(data.cpu(), corr_array=corr_array)

        with torch.no_grad():
          outputs = self.model(data)

        _, preds = torch.max(outputs, 1)

        running_corrects, running_all = self._update_attr(running_corrects, running_all, preds, gender_labels.data, t)
        # corrects = torch.sum(preds == label.data)

        if i % 400 == 0:
          # print("Iteration {}".format(i))
          print("*"*20)

        del data, gender_labels, preds

      accs = []
      for i in self.attr:
        acc = running_corrects[i]/running_all[i]
        accs.append(acc)
        print("Group {} Test Accuracy = {:.4f}, all images = {}".format(i, acc, running_all[i]))

      print(accs)
      print(sum(running_corrects.values()) / sum(running_all.values()))
      if save_path is not None:
        np.save(save_path, np.array(accs))
      return running_corrects, running_all, accs

    def _update_attr(self, running_corrects, running_all, preds, label, t):
      for i in range(len(t)):
        running_all[t[i]] += 1
        if preds[i] == label[i]:
          running_corrects[t[i]] += 1
      return running_corrects, running_all
    
    def eval_(self):
      self.model.eval()
      race_running_corrects = 0
      gender_running_corrects = 0
      age_running_corrects = 0
      running_all = 0

      for i, (data, a, label, b, t) in enumerate(self.val_dataloader):

        torch.cuda.empty_cache()

        data = data.to(device)
        label = label.squeeze().to(device)

        with torch.no_grad():
          outputs = self.model(data)

        gender_label = label

        # race_score = outputs[:, :7].data
        # _, race_preds = torch.max(race_score, 1)

        gender_score = outputs
        _, gender_preds = torch.max(gender_score, 1)

        # age_score = outputs[:, 9:18].data
        # _, age_preds = torch.max(age_score, 1)
        
        # race_label = label[:, 0]
        # gender_label = label[:, 1]
        # age_label = label[:, 2]

        # race_corrects = torch.sum(race_preds == race_label.data)
        gender_corrects = torch.sum(gender_preds == gender_label.data)
        # age_corrects = torch.sum(age_preds == age_label.data)

        # race_running_corrects += torch.sum(race_corrects)
        gender_running_corrects += torch.sum(gender_corrects)
        # age_running_corrects += torch.sum(age_corrects)

        running_all += len(data)

        # if i % 10 == 0:
        #   print("Iteration {}".format(i), "correct preded: {} {} {}".format(gender_corrects))
        #   print("*"*20)

        del data, label

      # race_acc = race_running_corrects/running_all
      gender_acc = gender_running_corrects/running_all
      # age_acc = age_running_corrects/running_all

      # print("Race Test Accuracy = {:.4f}, Gender Test Accuracy = {:.4f}, Age Test Accuracy = {:.4f}".format(race_acc, gender_acc, age_acc))
      print("Gender Test Accuracy = {:.4f}".format(gender_acc))
      # return (race_acc, gender_acc, age_acc)

    def get_perturbation(self):
      print("Start Attack..")
      advs = {i:[] for i in self.attr}
      adv_dist = {i:[] for i in self.attr}
      self.model.eval().to(device)

      fmodel = foolbox.models.PyTorchModel(self.model.eval(), bounds=(-3, 3), num_classes=2)
      for i, (data, a, labels, c, attr) in enumerate(self.val_dataloader):

        inputs = np.array(data)
        labels = np.array(labels.squeeze())
        
        attack = foolbox.attacks.FGSM(fmodel, distance=foolbox.distances.Linf)
        adversarials = attack(inputs, labels, unpack=False)
        # adversarials = attack(inputs, labels[:,1], unpack=False)

        advs, adv_dist = self._add_advs(advs, inputs, adversarials, adv_dist, attr)

        if i % 10 == 0:
          print("checkpoint {}".format(i))
      return advs, adv_dist

    def _add_advs(self, advs, data, adversarials, adv_dist, t):
      for i in range(len(t)):
        if adversarials[i].perturbed is not None and adversarials[i].distance.value!= 0:
          advs[t[i]].append(adversarials[i].perturbed - data[i])
          adv_dist[t[i]].append(adversarials[i].distance.value)
      return advs, adv_dist

    def run(self, epochs):
      print("Start Training...")
      self.metrics = []
      running_loss = 0
      running_corrects = 0
      for e in range(epochs):
        epoch_loss = 0
        epoch_loss = 0
        correct = 0
        # self.eval_()
        self.model.train()
        
        print('Epoch-{0} lr: {1}'.format(e, optimizer.param_groups[0]['lr']))
        for i, (data, race_labels, gender_labels, age_labels, attr) in enumerate(self.train_dataloader):
          self.optimizer.zero_grad()
          
          data = data.to(device)
          race_labels = race_labels.squeeze().to(device)
          gender_labels = gender_labels.squeeze().to(device)
          age_labels = age_labels.squeeze().to(device)
          
          outputs = self.model(data)
          _, preds = torch.max(outputs, 1)

          # change later
          if self.dataset == 'fairface':
            race_outputs = outputs[:,:7]
            gender_outputs = outputs[:,7:9]
            age_outputs = outputs[:,9:18]

            loss0 = self.criterion(race_outputs, race_labels)
            loss1 = self.criterion(gender_outputs, gender_labels)
            loss2 = self.criterion(age_outputs, age_labels)

          if self.dataset == 'utkface':
            # different label order with fairface 
            race_outputs = outputs[:,:5]
            gender_outputs = outputs[:,5:7]
            age_outputs = outputs[:,7:]

            loss0 = self.criterion(race_outputs, age_labels)
            loss1 = self.criterion(gender_outputs, gender_labels)
            loss2 = self.criterion(age_outputs, race_labels)

          # race_labels = label[:,0]
          # gender_labels = label[:,1]
          # age_labels = label[:,2]

          loss = loss0 + loss1 + loss2
          running_loss += loss.item()
          loss.backward()
          self.optimizer.step()

          del data, outputs, preds
          if i % 600 == 0:
            print("Iteration {}, Race loss {:.4f}, Gender loss {:.4f}, Age loss {:.4f}".format(i, loss0.item(), loss1.item(), loss2.item()))
            print("*"*20)
        if self.scheduler is not None:
          self.scheduler.step()
        
      self.save_model(self.save_path)

In [ ]:
# models = [
#       "Fairface_Multi_layer0_5_layerall_3_epoch_13_trial1.pt",
#       "Fairface_Multi_layer0_3_layerall_3_epoch_13_trial1.pt",
#       "Fairface_Multi_layer0_7_layerall_3_epoch_13_trial1.pt",
#       "Fairface_Multi_layer0_11_layerall_3_epoch_13_trial1.pt",
#       "Fairface_Multi_layer0_13_layerall_3_epoch_13_trial1.pt",
#       "FairFace_ViT_epoch_13_trial1.pt"
#       ]
models = ['UTKFace_Multi_layer0_11_layerall_3_epoch_13_trial1.pt']

for i in [-1]:
  print("-"*50,"test", i, "-"*50)
  res = []
  for model in models:
    resnet34 = torch.load("models/{}".format(model))
    resnet34 = Fair_Face_Model_end_with_sigmoid(resnet34)
    optimizer = torch.optim.Adam(resnet34.parameters(), lr=0.002)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    My_trainer = Trainer(resnet34, optimizer=optimizer, scheduler=scheduler, train_dataloader=train_dataloader, val_dataloader=val_dataloader, attr='race', dataset='utkface')
    # cay = np.load("injection_corr/injection_corr_{}_8_dir.npy".format(i))
    _, _, accs = My_trainer.eval_attr(modified=False, corr_array=None)
    res.append(accs)
  np.save("results_acc/injection_acc_{}_8_dir_trial1.npy".format(i), res)

In [ ]:
model = 'Fairface_Multi_layer0_13_layerall_3_epoch_13_trial1.pt'
resnet34 = torch.load("models/{}".format(model))
resnet34 = Fair_Face_Model_end_with_sigmoid(resnet34)
optimizer = torch.optim.Adam(resnet34.parameters(), lr=0.002)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
My_trainer = Trainer(resnet34, optimizer=optimizer, scheduler=scheduler, train_dataloader=train_dataloader, val_dataloader=val_dataloader, attr='race', dataset='fairface')
c = My_trainer.get_confusion()

Train

In [ ]:
for config in configs:
  resnet34 = tv.resnet34(pretrained=False)
  resnet34 = config["layer_function"](resnet34)
  resnet34.apply(weights_init)
  optimizer = torch.optim.Adam(resnet34.parameters(), lr=0.002)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
  My_trainer = Trainer(resnet34, optimizer=optimizer, scheduler=scheduler, train_dataloader=train_dataloader, val_dataloader=val_dataloader, attr='race', save_path=config["save_path"], dataset='fairface')
  My_trainer.run(13)

Start Training...
Epoch-0 lr: 0.002


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Iteration 0, Race loss 2.1976, Gender loss 0.7081, Age loss 2.5971
********************
Iteration 600, Race loss 1.6785, Gender loss 0.5782, Age loss 1.7416
********************
Epoch-1 lr: 0.002
Iteration 0, Race loss 1.6997, Gender loss 0.4338, Age loss 1.7650
********************
Iteration 600, Race loss 1.4266, Gender loss 0.2938, Age loss 1.4990
********************
Epoch-2 lr: 0.002
Iteration 0, Race loss 1.5721, Gender loss 0.3890, Age loss 1.3565
********************
Iteration 600, Race loss 1.2022, Gender loss 0.3581, Age loss 1.2916
********************
Epoch-3 lr: 0.002
Iteration 0, Race loss 1.2857, Gender loss 0.2596, Age loss 1.2015
********************
Iteration 600, Race loss 1.0103, Gender loss 0.2934, Age loss 1.1791
********************
Epoch-4 lr: 0.002
Iteration 0, Race loss 1.0733, Gender loss 0.2983, Age loss 1.2730
********************
Iteration 600, Race loss 1.0712, Gender loss 0.2313, Age loss 1.1808
********************
Epoch-5 lr: 0.001
Iteration 0, Race lo

Inference

In [ ]:
models = [
      # "Fairface_Multi_layer0_3_layerall_3_epoch_13_trial2.pt",
      "Fairface_Multi_layer0_7_layerall_7_epoch_13_trial2.pt",]

folder_names = [
        # 'Fairface_layer0_3_layerall_3_att_2',
        'Fairface_layer0_7_layerall_7_att_2']

ages = [ '0-2', 
      '3-9',
      '10-19',
      '20-29',
      '30-39',
      '40-49',
      '50-59',
      '60-69',
      'more than 70',
      ]

races = ['East Asian',
      'White',
      'Latino_Hispanic',
      'Southeast Asian',
      'Black',
      'Indian',
      'Middle Eastern']

In [ ]:
for model, f in zip(models, folder_names):
  resnet34 = torch.load("models/{}".format(model))
  resnet34 = Fair_Face_Model_end_with_sigmoid(resnet34)
  optimizer = torch.optim.Adam(resnet34.parameters(), lr=0.002)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
  My_trainer = Trainer(resnet34, optimizer=optimizer, scheduler=scheduler, train_dataloader=train_dataloader, val_dataloader=val_dataloader, attr='race', dataset='fairface')
  advs, adv_dist = My_trainer.get_perturbation()
  folder_name = f
  os.makedirs("attack_perturbation/race/{}".format(folder_name))
  for i in races:
    if len(advs[i]) > 2000:
      advs[i] = advs[i][:2000]
    np.save("attack_perturbation/race/{}/{}_race_trial2.npy".format(folder_name, i), advs[i])
    np.save("attack_perturbation/race/{}/{}_race_dist_trial2.npy".format(folder_name, i), adv_dist[i])

Start Attack..


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


checkpoint 0
checkpoint 10
checkpoint 20
checkpoint 30
checkpoint 40
checkpoint 50
checkpoint 60
checkpoint 70
checkpoint 80
checkpoint 90
checkpoint 100
checkpoint 110
checkpoint 120
checkpoint 130
checkpoint 140
checkpoint 150
checkpoint 160
checkpoint 170


In [ ]:
resnet34 = torch.load("models/Fairface_Multi_layer0_11_layerall_3_epoch_13_trial1.pt")
resnet34 = Fair_Face_Model_end_with_sigmoid(resnet34)

In [ ]:
optimizer = torch.optim.Adam(resnet34.parameters(), lr=0.002)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
My_trainer = Trainer(resnet34, optimizer=optimizer, scheduler=scheduler, train_dataloader=train_dataloader, val_dataloader=val_dataloader, attr='age', dataset='fairface')

In [ ]:
advs, adv_dist = My_trainer.get_perturbation()

Start Attack..


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


checkpoint 0
checkpoint 10
checkpoint 20
checkpoint 30
checkpoint 40
checkpoint 50
checkpoint 60
checkpoint 70
checkpoint 80
checkpoint 90
checkpoint 100
checkpoint 110
checkpoint 120
checkpoint 130
checkpoint 140
checkpoint 150
checkpoint 160
checkpoint 170


In [ ]:
advs.keys()

dict_keys(['East Asian', 'White', 'Latino_Hispanic', 'Southeast Asian', 'Black', 'Indian', 'Middle Eastern'])

In [ ]:
ages = [ '0-2', 
      '3-9',
      '10-19',
      '20-29',
      '30-39',
      '40-49',
      '50-59',
      '60-69',
      'more than 70',
      ]

races = ['East Asian',
      'White',
      'Latino_Hispanic',
      'Southeast Asian',
      'Black',
      'Indian',
      'Middle Eastern']

tmp = ['dark', 'light']

folder_name = 'Fairface_layer0_11_layerall_3_att_1'
# os.makedirs("attack_perturbation/age/{}".format(folder_name))
for i in ages:
  if len(advs[i]) > 2000:
    advs[i] = advs[i][:2000]
  np.save("attack_perturbation/age/{}/{}_age_trial1.npy".format(folder_name, i), advs[i])
  np.save("attack_perturbation/age/{}/{}_age_dist_trial1.npy".format(folder_name, i), adv_dist[i])

In [ ]:
! ls

 attack_perturbation		       models
 configs.py			       Multi_task_train.ipynb
'Copy of Multi_task_train.ipynb'       __pycache__
 Fairface_layer0_11_layerall_3_att_1   utils.py


In [ ]:
corrects

{'Black': {0: 688, 1: 712},
 'East Asian': {0: 744, 1: 730},
 'Indian': {0: 702, 1: 719},
 'Latino_Hispanic': {0: 714, 1: 773},
 'Middle Eastern': {0: 744, 1: 356},
 'Southeast Asian': {0: 663, 1: 628},
 'White': {0: 1009, 1: 845}}

In [ ]:
all

{'Black': {0: 807, 1: 804},
 'East Asian': {0: 810, 1: 785},
 'Indian': {0: 763, 1: 763},
 'Latino_Hispanic': {0: 776, 1: 817},
 'Middle Eastern': {0: 785, 1: 388},
 'Southeast Asian': {0: 735, 1: 680},
 'White': {0: 1109, 1: 932}}

In [ ]:
CUDA_LAUNCH_BLOCKING=1
val_transforms = transforms.Compose([
            # transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
input = Image.open('../skintone/04_0003_A0_C1_H1_G0.jpg')
input = val_transforms(input)
# input.cuda()
# resnet34 = tv.resnet34(pretrained=False).cuda()
resnet34.cuda()(input.reshape(1, 3, 224, 224).cuda())

tensor([[ 0.0128,  0.3661, -0.4858,  0.6124, -0.3701, -0.1673,  0.8948,  0.2307,
         -1.5051, -0.2465,  1.1135,  0.6773, -0.9087,  0.2417,  0.0015,  0.0148,
         -0.2433,  0.6436]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [23]:
import csv

with open("../utkface/utkface.csv", "r") as file:
  csv_reader = csv.reader(file, delimiter=',')
  race_d = {i:0 for i in ['0','1','2','3','4']}

  for row in csv_reader:
    for r in row:
      race_d[r[-3:-2]]+=1
print(race_d)
  

{'0': 10078, '1': 4526, '2': 3434, '3': 3975, '4': 1692}
